## 3.1 计算图

### 1. 在不同计算图上定义和使用张量

In [1]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer()) #书中为tf.zeros_initializer(shape=[1]),此方法已弃用
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))        

[ 0.]
[ 1.]


## 3.2 张量

In [2]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b
print("result:", result)

sess = tf.InteractiveSession ()
print("result.eval()", result.eval())
sess.close()

result: Tensor("add:0", shape=(2,), dtype=float32)
result.eval() [ 3.  5.]


## 3.3 会话

### 1. 计算张量的取值

In [3]:
#方法1
sess = tf.Session()
with sess.as_default():
    print(result.eval())
    
#方法2
sess = tf.Session()
print(sess.run(result))

#方法3
print(result.eval(session=sess))

[ 3.  5.]
[ 3.  5.]
[ 3.  5.]


### 2. 自动生成默认会话

In [4]:
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[ 3.  5.]


### 3. GPU运算放到CPU运行

In [5]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

## 3.4 神经网络

### 1. 前向传播

In [10]:
w1 = tf.Variable(tf.random_normal((2, 3), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((3, 1), stddev=1, seed=1))

x = tf.constant([[0.7, 0.9]])

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()

sess.run(w1.initializer)
sess.run(w2.initializer)

print(sess.run(y))
sess.close()

[[ 3.95757794]]


### 2. 完整神经网络样例程序

In [15]:
from numpy.random import RandomState

batch_size = 8

w1 = tf.Variable(tf.random_normal((2, 3), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((3, 1), stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name="input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

#前向传播
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

#损失函数（交叉熵）和反向传播
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)) + (1-y) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)) ) 
train_step = tf.train.AdadeltaOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

with tf.Session( ) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        
        if i % 1000 == 0:
            total_loss_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print(i, total_loss_entropy)
   
    #训练之后的参数    
    print(sess.run(w1))
    print(sess.run(w2))


[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
0 0.314668
1000 0.314259
2000 0.313718
3000 0.313099
4000 0.312431
[[-0.8152228   1.4884994   0.06919771]
 [-2.44705534  0.10362051  0.59560919]]
[[-0.81617254]
 [ 1.48879337]
 [ 0.06990979]]
